## Modelo GMM

In [ ]:
# @title 1. Importações e Configurações Iniciais

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.mixture import GaussianMixture
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import (
    average_precision_score,
    precision_recall_curve,
    classification_report,
    confusion_matrix
)

# Configurações
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

DATA_PATH = 'data/processed'
OUTPUT_PATH = 'outputs'

if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

print("Bibliotecas importadas e diretórios configurados.")

Mounted at /content/drive


In [ ]:
# @title 2. Função de Carregamento e Preparação dos Dados

def load_data(data_path):
    # Carrega os dados (certifique-se que os arquivos existem no caminho)
    try:
        X_train = pd.read_csv(os.path.join(data_path, 'X_train_processed.csv'))
        y_train = pd.read_csv(os.path.join(data_path, 'y_train.csv'))['Class']

        X_test = pd.read_csv(os.path.join(data_path, 'X_test_processed.csv'))
        y_test = pd.read_csv(os.path.join(data_path, 'y_test.csv'))['Class']
        ids_test = pd.read_csv(os.path.join(data_path, 'ids_test.csv'))['id']
        
        # Treino APENAS com normais (Semi-supervisionado)
        X_train_normal = X_train[y_train == 0]
        
        return X_train_normal, X_test, y_test, ids_test
    except FileNotFoundError as e:
        print(f"Erro ao carregar arquivos: {e}")
        return None, None, None, None

In [ ]:
# @title 3. Execução do Carregamento

X_train_normal, X_test, y_test, ids_test = load_data(DATA_PATH)

if X_train_normal is not None:
    print(f"Dados Carregados com Sucesso:")
    print(f"Treino (Apenas Normais): {X_train_normal.shape}")
    print(f"Teste (Normais + Fraudes): {X_test.shape}")
else:
    print("Falha no carregamento dos dados.")

In [ ]:
# @title 4. Função de Treinamento e Avaliação

def train_and_evaluate_gmm(params, X_train, X_test, y_test):
    # Instancia o modelo
    gmm = GaussianMixture(
        n_components=params['n_components'],
        covariance_type=params['covariance_type'],
        random_state=RANDOM_SEED
    )
    
    # Treina apenas com dados normais
    gmm.fit(X_train)
    
    # Avalia no conjunto de teste (Score: Log-likelihood negativo)
    # Quanto menor o log-likelihood, maior a chance de ser anomalia
    # Multiplicamos por -1 para que scores ALTOS sejam anomalias
    scores = -gmm.score_samples(X_test)
    
    auc_pr = average_precision_score(y_test, scores)
    
    return auc_pr, gmm, scores

In [ ]:
# @title 5. Definição dos Hiperparâmetros (Tunagem ou Fixo)

# 0 = Execução normal (apenas melhores hiperparâmetros)
# 1 = Grid Search (tunagem completa)
RUN_TUNING = 0 

if RUN_TUNING:
    print(">>> MODO: GRID SEARCH ATIVADO")
    param_grid = {
        'n_components': [1, 2, 3, 4],
        'covariance_type': ['full', 'diag']
    }
else:
    print(">>> MODO: EXECUÇÃO ÚNICA (MELHORES PARÂMETROS)")
    # Melhores parâmetros identificados anteriormente
    param_grid = {
        'n_components': [3],
        'covariance_type': ['full']
    }

grid = list(ParameterGrid(param_grid))
print(f"Total de combinações a testar: {len(grid)}")

In [ ]:
# @title 6. Execução do Treinamento

best_auc_pr = -1
best_model = None
best_params = None
best_scores = None

print("=============================================")
print(f"INICIANDO EXECUÇÃO")
print("=============================================")

for i, params in enumerate(grid):
    print(f"[{i+1}/{len(grid)}] Testando: {params} ...", end=" ")

    try:
        auc_pr, model, scores = train_and_evaluate_gmm(
            params,
            X_train_normal,
            X_test,
            y_test
        )
        print(f"AUC-PR: {auc_pr:.4f}")

        # Salva o melhor modelo
        if auc_pr > best_auc_pr:
            best_auc_pr = auc_pr
            best_model = model
            best_params = params
            best_scores = scores # Scores do melhor modelo no conjunto de teste

    except Exception as e:
        print(f"Erro: {e}")

print("\n🏆 MELHOR RESULTADO ENCONTRADO")
print(f"Parâmetros: {best_params}")
print(f"AUC-PR Final: {best_auc_pr:.4f}")

In [ ]:
# @title 7. Avaliação Final e Exportação dos Resultados

if best_model is not None:
    # 1. Recalcular scores finais (garantia)
    final_scores = -best_model.score_samples(X_test)
    
    # 2. Definir Threshold para Recall ~0.80
    precision, recall, thresholds = precision_recall_curve(y_test, final_scores)
    
    target_recall = 0.80
    # Encontra o índice onde o recall é mais próximo de 0.80
    idx = (np.abs(recall - target_recall)).argmin()
    final_threshold = thresholds[idx]
    
    print(f"🎯 Threshold escolhido: {final_threshold:.6f}")
    print(f"   Recall esperado: {recall[idx]:.4f}")
    print(f"   Precision esperada: {precision[idx]:.4f}")
    
    # 3. Gerar predições binárias
    y_pred = (final_scores >= final_threshold).astype(int)
    
    print("\n--- RELATÓRIO DE CLASSIFICAÇÃO ---")
    print(classification_report(y_test, y_pred, target_names=['Normal', 'Fraude']))
    
    print("Matriz de Confusão:")
    print(confusion_matrix(y_test, y_pred))

    # 4. Salvar CSV de predições
    results_df = pd.DataFrame({
        'id': ids_test,
        'anomaly_score': final_scores,
        'is_anomaly': y_pred
    })
    
    csv_path = os.path.join(OUTPUT_PATH, 'gmm_predictions.csv')
    results_df.to_csv(csv_path, index=False)
    print(f"\n✅ Arquivo salvo em: {csv_path}")
else:
    print("Erro: Nenhum modelo foi treinado.")